In [1]:
"""
This is the first part of TensorFlow tutorial
The idea is to show the basic computation patterns, 
and write simple models for hand-written digit classification on MNIST dataset
"""
import tensorflow as tf
import numpy as np

/Users/tieqiao1024/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/tieqiao1024/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
## Section 1: TF basics
#  1.1 Graphs and sessions
A = tf.Variable(2)
B = tf.Variable(3)
C = A+B
print (C)
## TODO: Write a simple calculation C=A+B

Tensor("add:0", shape=(), dtype=int32)


In [ ]:
## Build a session to execute the computation
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print (sess.run(C))

In [ ]:
## close your previous session
## and clear its information
tf.reset_default_graph()
sess.close()

## Below is how you use an interactive session
## Can you see the difference?
sess = tf.InteractiveSession()

tf.global_variables_initializer().run()
print (C.eval())

In [ ]:
## close your previous session
## and clear its information
tf.reset_default_graph()
sess.close()

In [ ]:
## Section 1: Building a model with TF
from tensorflow.examples.tutorials.mnist import input_data
file = "./MNIST"
mnist = input_data.read_data_sets(file, one_hot=True)

In [ ]:
# Task 1
# Please read and run this example, 
# And see a minimal instance of TensorFlow
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
y = tf.nn.softmax(tf.matmul(x, W) + b)
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
for i in range(300):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [ ]:
# Use the testing set to see the model accuracy
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_:mnist.test.labels}))
sess.close()
tf.reset_default_graph()

In [ ]:
# Task 2
# Now try by yourself to build a 2-conv + 1FC layer

sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.global_variables_initializer())

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
# TODO: complete the model
# Write your output as y_conv
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
sess.run(tf.global_variables_initializer())
for i in range(300):
    batch = mnist.train.next_batch(100)
    if i%10 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))

    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    #print("test accuracy %g"%accuracy.eval(feed_dict={
    #    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

In [ ]:
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
sess.close()

In [ ]:
sess.close()

In [ ]:
# Task 2.3
# Some "layer" API calls brought by TF
# This is not finished
sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)
sess.run(tf.global_variables_initializer())

x_image = tf.reshape(x, [-1,28,28,1])
conv1=tf.layers.conv2d(inputs=x_image, kernel_size=[5,5], filters=32)
pool1=tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=1)

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
sess.run(tf.global_variables_initializer())
for i in range(300):
    batch = mnist.train.next_batch(100)
    if i%10 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))

    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    #print("test accuracy %g"%accuracy.eval(feed_dict={
    #    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

In [ ]:
# Again, test the accuracy of this model
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
sess.close()